# ループである地域範囲の住所データを取得

## ライブラリのインポート、API設定、関数定義

In [98]:
import requests
import json
import pandas as pd
import time
import numpy as np


# Google PlaceのAPIキーを入力
api_key = '認証したAPIキーを入力'


#緯度と経度と地点分割粒度から最適な検索半径を抽出する関数
def set_radius(lat_range, lng_range, lat_seg_num, lng_seg_num):
    if (lat_range[1] - lat_range[0])  * 110904 / ((lat_seg_num -1) ) >= (lng_range[1] - lng_range[0])  * 93452 / ((lng_seg_num -1) ):
        return np.round((lat_range[1] - lat_range[0])  * 110904 * np.sqrt(2) / ((lat_seg_num -1) * 2), -2)
    else:
        return np.round((lng_range[1] - lng_range[0])  * 93452 * np.sqrt(2) / ((lng_seg_num -1) * 2), -2)


#条件から周辺施設を検索する関数
def Get_GioData(location, keyword, api_key, radius=50000):
    result = []
    next_page_token = None
    while True:
        if next_page_token:
            time.sleep(3)
            url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location}&radius={radius}&keyword={keyword}&language=ja&key={api_key}&pagetoken={next_page_token}'
        else:
            time.sleep(3)
            url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location}&radius={radius}&keyword={keyword}&language=ja&key={api_key}'
        response = requests.get(url)
        data = response.json()
        result += data['results']
        next_page_token = data.get('next_page_token')
        if not next_page_token:
            break
    return result

## コーディング



###Case1.   　経度,緯度の範囲から検索施設情報の検索（１単語）





　　　1.1   パラメーター設定



In [74]:
#今回の検索したい施設のキーワードを入力
keyword = 'コンビニ'

#福岡市おおよその緯度と経度を指定(URL:https://nlftp.mlit.go.jp/cgi-bin/isj/dls/_choose_method.cgi)のデータを参照
lat_range = [33.45, 33.7]
lng_range = [130.22, 130.47]

#緯度方向,経度方向にそれぞれ何分割してサンプリングするかを指定(数が大きいほど細かくサンプリングできるが、API使用料金の上限に注意する必要あり)
lat_seg_num= 6
lng_seg_num = 6

1800.0




　　　1.2   データ抽出



In [18]:
#パラメーターを関数に渡し検索地点からの検索範囲を最適化
radius = set_radius(lat_range, lng_range, lat_seg_num, lng_seg_num)

#緯度,経度の範囲地点を配列に格納
lat_grid, lng_grid = np.meshgrid(np.linspace(lat_range[0], lat_range[1], num=lat_seg_num), np.linspace(lng_range[0], lng_range[1], num=lng_seg_num))


#データ抽出の格納先
result_dic_data = []

#各地点ごとにループを回して検索をかけていく
for i in range(lat_grid.shape[0]):
    for j in range(lng_grid.shape[1]):
        location = str(lat_grid[i,j]) + ',' + str(lng_grid[i,j])
        #必要なパラメータを関数に入力しデータ抽出
        results = Get_GioData(location, keyword, api_key, radius)

    #抽出データから必要なカラムを辞書形式で取得   
        for result in results:
            place = {}
            place["name"] = result["name"]
            place["address"] = result["vicinity"]
            place["lat"] = result["geometry"]["location"]["lat"]
            place["lng"] = result["geometry"]["location"]["lng"]
            place['rating'] = result['rating']
            place['user_ratings_total'] = result['user_ratings_total']
            result_dic_data.append(place)

#データフレームに変換
df = pd.DataFrame(result_dic_data)

#住所情報から市と区のカラムを取得
df["city"] = df["address"].apply(lambda x: x.split("市")[0] if "市" in x else None)
df["district"] = df["address"].apply(lambda x: x.split("区")[0].split("市")[-1] if "区" in x else None)
df['city-district'] = '福岡市' + df['district'] + '区'
df['search_word'] = keyword

#検索で出た重複を排除
df = df.drop_duplicates('name').reset_index(drop=True)


#キーワードごとにcsvファイルに出力
df.to_csv('福岡市周辺の' + keyword + 'に関する位置情報.csv')


###Case2.   　経度,緯度の範囲から検索施設情報の検索（複数単語）



2.1   パラメーター設定



In [7]:
#今回の検索したい施設のキーワードをリスト形式で入力
keyword_list = ['ジム', '動物病院' ]

#福岡市おおよその緯度と経度を指定(URL:https://nlftp.mlit.go.jp/cgi-bin/isj/dls/_choose_method.cgi)のデータを参照
lat_range = [33.45, 33.7]
lng_range = [130.22, 130.47]

#緯度方向,経度方向にそれぞれ何分割してサンプリングするかを指定(数が大きいほど細かくサンプリングできるが、API使用料金の上限に注意する必要あり)
lat_seg_num= 6
lng_seg_num = 6



2. 2   データ抽出



In [6]:

# 検索キーワードについて福岡市全域で検索をかけてデータを集計していく
for word in keyword_list:
    #検索したデータの格納先
    result_dic_data = []
    #各地点ごとにループを回して検索をかけていく
    for i in range(lat_grid.shape[0]):
        for j in range(lng_grid.shape[1]):
            location = str(lat_grid[i,j]) + ',' + str(lng_grid[i,j])
            #必要なパラメータを関数に入力しデータ抽出
            results = Get_GioData(location, word, api_key, radius)

        #抽出データから必要なカラムを辞書形式で取得   
            for result in results:
                place = {}
                place["name"] = result["name"]
                place["address"] = result["vicinity"]
                place["lat"] = result["geometry"]["location"]["lat"]
                place["lng"] = result["geometry"]["location"]["lng"]
                place['rating'] = result['rating']
                place['user_ratings_total'] = result['user_ratings_total']
                result_dic_data.append(place)

    #データフレームに変換
    df = pd.DataFrame(result_dic_data)

    #住所情報から市と区のカラムを取得
    df["city"] = df["address"].apply(lambda x: x.split("市")[0] if "市" in x else None)
    df["district"] = df["address"].apply(lambda x: x.split("区")[0].split("市")[-1] if "区" in x else None)
    df['city-district'] = '福岡市' + df['district'] + '区'
    df['search_word'] = word

    #検索で出た重複を排除
    df = df.drop_duplicates('name').reset_index(drop=True)

    #キーワードごとにcsvファイルに出力
    df.to_csv('福岡市周辺の' + word + 'に関する位置情報.csv')

###Case3.   　地点名称周辺の施設情報検索（１地点）



3.1   地点名から緯度,経度を取得する関数を定義



In [92]:
#地点名から緯度と経度を取得する関数
def Get_GioCode(place, api_key):
    url = 'https://maps.googleapis.com/maps/api/geocode/json?address={}&key={}'.format(place, api_key)
    response = requests.get(url)
    data = json.loads(response.text)
    latitude = data['results'][0]['geometry']['location']['lat']
    longitude = data['results'][0]['geometry']['location']['lng']
    location = str(latitude) + ',' + str(longitude)
    return location



3.2   パラメータの設定




In [ ]:
#検索地点の設定
place = '博多駅' 

#検索地点からの検索範囲を設定(m)
radius = 5000 

#今回の検索したい施設のキーワードを入力
keyword = 'ジム'



3.3  データの抽出 



In [ ]:
#検索した後に整形した必要なデータの格納先
result_dic_data = []

# 博多駅の座標を取得
location = Get_GioCode(place, api_key)

#関数にパラメータを渡して検索データを取得
results = Get_GioData(location, keyword, api_key, radius)

#抽出データから必要なカラムを辞書形式で取得   
for result in results:
    place = {}
    place["name"] = result["name"]
    place["address"] = result["vicinity"]
    place["lat"] = result["geometry"]["location"]["lat"]
    place["lng"] = result["geometry"]["location"]["lng"]
    place['rating'] = result['rating']
    place['user_ratings_total'] = result['user_ratings_total']
    result_dic_data.append(place)

#データフレームに変換
df = pd.DataFrame(result_dic_data)

#住所情報から市と区のカラムを取得
df["city"] = df["address"].apply(lambda x: x.split("市")[0] if "市" in x else None)
df["district"] = df["address"].apply(lambda x: x.split("区")[0].split("市")[-1] if "区" in x else None)
df['city-district'] = '福岡市' + df['district'] + '区'
df['search_word'] = keyword

#検索で出た重複を排除
df = df.drop_duplicates('name').reset_index(drop=True)

#csvファイルに出力
df.to_csv('福岡市周辺の' + keyword + 'に関する位置情報.csv')

## Case4 地点名称周辺の施設情報検索(複数地点)



4.1   パラメータの設定



In [ ]:
#検索地点の設定
location_name = ['博多駅', '西鉄天神駅','西鉄大橋駅','福岡自動車運転免許試験場','南福岡特別支援学校','福工大前駅','福岡市 貝塚駅',
                 '博多南駅','マリンワールド' ,'唐人町駅','ビッグモーター西福岡店', '福岡市立早良中学校','福岡市 青葉公園','福岡市 西部運動公園',
                 '姪浜駅', '九大学研都市駅', '福大前駅', '福岡海釣り公園', '博多の森','南福岡駅' ] 

#検索地点からの検索範囲を設定(m)
radius = 5000 

#今回の検索したい施設のキーワードを入力
keyword = 'ジム'



4.2  データの抽出 



In [ ]:
#検索した後に整形した必要なデータの格納先
result_dic_data = []

# 博多駅の座標を取得
for locate in location_name:
    location = Get_GioCode(locate, api_key)

    results = Get_GioData(location, keyword, api_key, radius)

#抽出データから必要なカラムを辞書形式で取得   
    for result in results:
        place = {}
        place["name"] = result["name"]
        place["address"] = result["vicinity"]
        place["lat"] = result["geometry"]["location"]["lat"]
        place["lng"] = result["geometry"]["location"]["lng"]
        place['rating'] = result['rating']
        place['user_ratings_total'] = result['user_ratings_total']
        result_dic_data.append(place)

#データフレームに変換
df = pd.DataFrame(result_dic_data)

#住所情報から市と区のカラムを取得
df["city"] = df["address"].apply(lambda x: x.split("市")[0] if "市" in x else None)
df["district"] = df["address"].apply(lambda x: x.split("区")[0].split("市")[-1] if "区" in x else None)
df['city-district'] = '福岡市' + df['district'] + '区'
df['search_word'] = keyword

#検索で出た重複を排除
df = df.drop_duplicates('name').reset_index(drop=True)

#csvファイルに出力
df.to_csv('福岡市周辺の' + keyword + 'に関する位置情報.csv')

## Appendix



1.   Googke Places APIで抽出できるデータ項目



In [ ]:
locate = 'キャナルシティ博多'
keyword = 'ラーメン'
radius = 10000
location = Get_GioCode(locate, api_key)
search_result = Get_GioData(location, keyword, api_key, radius=50000)
df = pd.DataFrame(search_result)
df['latitude'] = df['geometry'].apply(lambda x: x['location']['lat'])
df['longitude'] = df['geometry'].apply(lambda x: x['location']['lng'])
df.drop(columns=['geometry'], inplace=True)
df


 {'business_status': 'OPERATIONAL',
 
 'geometry': {'location': {'lat': 33.5928065, 'lng': 130.3855521},
 
  'viewport': {'northeast': {'lat': 33.59412092989271,
    'lng': 130.3871242798927},

   'southwest': {'lat': 33.59142127010727, 'lng': 130.3844246201072}}},
 
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png',

 'icon_background_color': '#7B9EB0',

 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet',

 'name': '福岡市中央区24時間営業パーソナルジム【VISION24 福岡天神店】',

 'opening_hours': {'open_now': True},

 'photos': [{'height': 3888,
   'html_attributions': ['<a href="https://maps.google.com/maps/contrib/102376585050495911523">A Google User</a>'],

'photo_reference':'AfLeUgNpwUPchx0Q45qhb0chkiJ9Cbe0IDNd0KvuEFGj807adaxSHcAwjkf4xl0aqxGOi4fFp4DY4YiYRdBaYihL3EwuhUHkhXMNW5LhZ1tEjTddF0ExMV94-K2Zqeu6KbeaR-kHaiNrZ1ZN1sD903ymlWrwaryhnKbWBiPO20htG9e7sDvF',
   'width': 5184}],

 'place_id': 'ChIJ_WZOWBGTQTUROlTqEPeYdjQ',

 'plus_code': {'compound_code': 'H9VP+46 福岡市, 日本、福岡県',
  'global_code': '8Q5GH9VP+46'},

 'rating': 4.8,

 'reference': 'ChIJ_WZOWBGTQTUROlTqEPeYdjQ',

 'scope': 'GOOGLE',

 'types': ['gym', 'health', 'store', 'point_of_interest', 'establishment'],

 'user_ratings_total': 269,

 'vicinity': '福岡市中央区港１丁目１０−５ キテラタウン長浜 ３F'


2.   GeoCodingAPIで取得できるデータ項目



In [ ]:
from pandas.io.json import json_normalize
place = '渋谷駅'
url = 'https://maps.googleapis.com/maps/api/geocode/json?address={}&key={}&language=ja'.format(place, api_key)
response = requests.get(url)
data = json.loads(response.text)
results = data['results']
df = json_normalize(results, sep='_')
df

In [ ]:
from pandas.io.json import json_normalize
results = data['results']
df = json_normalize(results, sep='_')
df